## Checking statistic for swimming
by Pontus

In [114]:
import pandas as pd
import plotly_express as px
import plotly.graph_objects as go
pd.set_option("display.max_rows", None)

In [115]:
directory_data = "../data"

athletes = pd.read_csv(f"{directory_data}/athlete_events.csv")
regions = pd.read_csv(f"{directory_data}/noc_regions.csv")

In [116]:
#athletes.info()
print(athletes.loc[athletes['Name'] == 'Jared Tallent', ['Age', 'Medal', 'Year']])

         Age   Medal  Year
235610  23.0  Bronze  2008
235611  23.0  Silver  2008
235612  27.0     NaN  2012
235613  27.0  Silver  2012
235614  31.0  Silver  2016


In [117]:
# Load file seperate fix, but since it was from excel convertion was different so needed to drop ; 
fixAge = pd.read_csv(f"{directory_data}/australia_missing_AGE_FIX.csv", sep=";")

# Drop first column if it's unnamed/empty
if fixAge.columns[0].startswith("Unnamed"):
    fixAge = fixAge.drop(columns=[fixAge.columns[0]])

# Merge only the corrected ages
merged_age = athletes.merge(
    fixAge[['ID', 'Age']],  # include ID or unique key + Age
    on='ID',
    how='left',
    suffixes=('', '_fixed')
)

# Replace missing Age values with fixed ones
merged_age['Age'] = merged_age['Age'].fillna(merged_age['Age_fixed'])

# Drop helper column
merged_age = merged_age.drop(columns=['Age_fixed'])

athletes = merged_age

# Save result
#merged_age.to_csv(f"{directory_data}/athlete_events_fixed_age.csv", index=False)

In [118]:
print(athletes.loc[athletes['Name'] == 'Jared Tallent', ['Age', 'Medal', 'Year']])

         Age   Medal  Year
235836  23.0  Bronze  2008
235837  23.0  Silver  2008
235838  27.0     NaN  2012
235839  27.0  Silver  2012
235840  31.0  Silver  2016


In [119]:
merged = pd.merge(athletes, regions, on="NOC", how="outer")
df = merged


In [120]:
print(df.loc[df['Name'] == 'Jared Tallent', ['Age', 'Medal', 'Year']])


        Age   Medal    Year
12050  23.0  Bronze  2008.0
12051  23.0  Silver  2008.0
12052  27.0     NaN  2012.0
12053  27.0  Silver  2012.0
12054  31.0  Silver  2016.0


In [121]:
df.at[11827, 'Medal'] = "Gold"
df = df[df['Year'] != 1906]
df = df[df['Sport'] != "Alpinism"]

In [122]:
swimming = df[df["Sport"] == "Swimming"]
swimming = swimming[swimming["Medal"].notna()]
swimming = swimming.drop_duplicates(["Year", "Event", "Season", "Medal", "NOC"])
medals = (swimming.groupby(["region", "Medal"])
        .size().unstack(fill_value=0)) 

medals["Total"] = medals["Gold"] + medals["Silver"] + medals["Bronze"] 
top10 = medals.sort_values(by="Total", ascending=False).head(10).reset_index()

medal_values = ["Bronze", "Silver", "Gold"]


fig = px.bar(
    top10,x=medal_values, y="region",
    color_discrete_map={"Gold":"#9F8F5E", "Silver": "#969696", "Bronze": "#996B4F"},
    labels={
        "region": "Country",
        "value": "Medal Count"
    },
    title="Top 10 countries in Swimming (Medals by type)"
)
fig.show()


In [ ]:
print(df[df['Year'].isnull()].shape)
print(df[df['Sport'] == 'Swimming'][df['Year'].isnull()])
missing_year_count = df['Sport'] == 'Swimming'][df['Year'].isnull().sum()
print(f"Rows with missing Year: {missing_year_count}")

"""
swimming = swimming.dropna(subset=['Year'])
nulls = df.isnull().sum()
print(nulls)"""


Rows with missing Year: Empty DataFrame
Columns: [ID, Name, Sex, Age, Height, Weight, Team, NOC, Games, Year, Season, City, Sport, Event, Medal, region, notes]
Index: []


C:\Users\Pontus\AppData\Local\Temp\ipykernel_21540\299277182.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



"\nswimming = swimming.dropna(subset=['Year'])\nnulls = df.isnull().sum()\nprint(nulls)"

In [136]:
swimming = df[df["Sport"] == "Swimming"]
swimming = swimming[swimming["Medal"].notna()]
swimming = swimming.dropna()
swimming = swimming.drop_duplicates(["Year", "Event", "Season", "Medal", "NOC"])

fig = px.sunburst(
    swimming,
    path=["Sport", "Year", "region", "Medal"],
    values=None,
    color="Medal",
    color_discrete_map={"Gold":"#9F8F5E", "Silver": "#969696", "Bronze": "#996B4F"},
    title="Swimming Medals by Year, Country, and Medal Type"
)
fig.show()

In [ ]:
AUS_medals = df[df['region'] == "Australia"]

AUS_nulls = AUS_medals.isnull().sum()
print(AUS_nulls)

ID           0
Name         0
Sex          0
Age          5
Height    1167
Weight    1296
Team         0
NOC          0
Games        0
Year         0
Season       0
City         0
Sport        0
Event        0
Medal     6593
region       0
notes     7857
dtype: int64


In [ ]:
"""df.at[211197, 'region'] = "Singapore"
age_map = {
    7584: 23,
    7947: 21,
    8083: 24,
    76826: 20,
    84171: 22,
    84172: 22,
    92526: 19,
    97005: 25,
    110696: 26,
    112232: 23,
    112414: 24,
    117528: 20,
    117899: 21,
    118521: 22,
    118917: 23,
    119552: 24,
    146697: 22,
    152228: 21,
}
df.loc[list(age_map.keys()), 'Age'] = pd.Series(age_map) """

In [ ]:
medals = df[df['Medal'].notna()]


In [ ]:
swimming_medals = medals[medals['Sport'] == "Swimming"]

swimming_nulls = swimming_medals.isnull().sum()
print(swimming_nulls)

ID           0
Name         0
Sex          0
Age         17
Height     469
Weight     548
Team         0
NOC          0
Games        0
Year         0
Season       0
City         0
Sport        0
Event        0
Medal        0
region       0
notes     3016
dtype: int64


In [ ]:
print(swimming_medals[swimming_medals['Medal'].isnull()])

Empty DataFrame
Columns: [ID, Name, Sex, Age, Height, Weight, Team, NOC, Games, Year, Season, City, Sport, Event, Medal, region, notes]
Index: []


In [ ]:
swimming_medals.iloc[210967]

IndexError: single positional indexer is out-of-bounds

In [ ]:


swimming_medalsgrouped = swimming_medals.groupby(["region", "Year", "Sport"])["Medal"].value_counts().reset_index(name="MedalCount")
#swimming_medalsgrouped